# Paralelização

Até agora o código que estamos construindo roda em série em nosso computador. Isso significa que cada instrução em nosso código é processada linearmente, uma após a outra.

Por exemplo, no caso de um `loop`, cada **etapa** do loop é executada em série. Vamos ver isso na prática.

## Códigos Seriais

Vamos começar analisando a **ordem de execução** de um loop simples:

In [1]:
import time

In [2]:
time.sleep(1)

In [3]:
def dormir(x):
    '''
    Dormir x-segundos e retorna x.
    '''
    
    print(f'Dormindo por {x} segundos.\n')
    time.sleep(x)
    print(f'Retornando {x}\n')
    return x

In [5]:
%%time
dormir(5)

Dormindo por 5 segundos.

Retornando 5

CPU times: user 4.37 ms, sys: 2.89 ms, total: 7.25 ms
Wall time: 5 s


5

Vamos construir uma lista e utilizar um `loop` para percorrer nossa lista:

In [6]:
my_list = [1,2,3,4]

In [20]:
%%time
for x in my_list:
    dormir(x)

Dormindo por 1 segundos.

Retornando 1

Dormindo por 2 segundos.

Retornando 2

Dormindo por 3 segundos.

Retornando 3

Dormindo por 4 segundos.

Retornando 4

CPU times: user 9.85 ms, sys: 3.51 ms, total: 13.4 ms
Wall time: 10 s


O `loop` levou 10s para percorrer a lista pois percorreu ela em **série**: elemento a elemento, um após o outro.

Mesmo as técnicas de *programação funcional* executam o código da mesma maneira:

In [8]:
%%time
list(map(dormir, [1, 2, 3, 4]))

Dormindo por 1 segundos.

Retornando 1

Dormindo por 2 segundos.

Retornando 2

Dormindo por 3 segundos.

Retornando 3

Dormindo por 4 segundos.

Retornando 4

CPU times: user 12.7 ms, sys: 4.4 ms, total: 17.1 ms
Wall time: 10 s


[1, 2, 3, 4]

## Códigos Paralelos

Uma das principais técnicas para aumentar a velocidade dos processadores no séc. XXI são os processadores com múltiplos *cores* (um processador contém múltiplas CPUs).

Cada *core* de um computador é linear: ele executa, em altissíma velocidade, instruções de forma linear - uma após a outra.

Os programas que construímos até hoje nas aulas são incapazes de utilizar múltiplos *cores* de processamento pois são lineares! Todos os `loops`, `applies`, `requests` ocorreram em série, de forma que, se nosso processador possui mais de um *core* não alavancamos toda sua capacidade computacional.

Vamos aprender agora como podemos converter nossos programas seriais em programas paralelos:

In [ ]:
!pip3 install multiprocess

In [9]:
from multiprocess import Pool, cpu_count


In [10]:
cpu_count()

8

A função `cpu_count()` nos mostra quantos *cores* nosso processador possui. Vamos utilizar a biblioteca `multiprocessing` para parelizar um loop.

## Criando um `Pool`

Um `Pool` é um objeto que coordena as tarefas que precisam ser executadas (*seu programa*) e como estas são alocadas nos diferentes *cores* do processador (*seu computador*).

Antes de mais nada, precisamos criar um `Pool`, determinando quantos *cores* esse *gerente* poderá utilizar.

In [11]:
pool = Pool(processes=cpu_count()-1)

Dormindo por 4 segundos.
Dormindo por 1 segundos.
Dormindo por 2 segundos.
Dormindo por 3 segundos.




Retornando 1

Retornando 2

Retornando 3

Retornando 4



In [12]:
%%time
my_list = [1, 2, 3, 4]
result = pool.map(dormir, my_list)
print(result)
pool.terminate()

[1, 2, 3, 4]
CPU times: user 23.6 ms, sys: 13.8 ms, total: 37.4 ms
Wall time: 4.05 s


O que aconteceu? O processamento **paralelo terminou em 4s**, comparado aos **10s do processamento em série**! O objeto `Pool` despachou cada uma das aplicações da função para um *core* diferente e as executou simultaneamente:

![image](images/parallel_vs_serial.webp)

### Nem todos os loops podem ser paralelos...

Para que um loop possa ser paralelizado, os resultados de cada *perna* do loop deve ser independente das outras *pernas*. Vamos construir um loop que não pode ser paralelizado:

In [13]:
minha_lista = [1, 2, 3, 4, 5]
fatorial = 1

for x in minha_lista:
    fatorial = fatorial * x

print(fatorial)

120


Cada *perna* do nosso `loop` **DEPENDE** da *perna* anterior! Logo, para executar a segunda etapa do `loop` precisamos executar a primeira, para executar a terceira, precisamos executar a segunda, e assim por diante.

Quais `loops` conseguimos paralelizar? Todos aqueles que podem ser escritos através de uma `list comprehensions` ou um `apply`.

## Utilizando funções `lambda` 

In [14]:
%%time
result = list(map(lambda x:x**10000000, [1,2,3,4,5,6]))

CPU times: user 14.2 s, sys: 77.8 ms, total: 14.3 s
Wall time: 14.3 s


In [18]:
%%time
pool = Pool(processes=cpu_count()-1)
result = pool.map(lambda x:x**10000000, [1,2,3,4,5,6,7,8])

CPU times: user 42.3 ms, sys: 79.6 ms, total: 122 ms
Wall time: 5.61 s


In [19]:
pool.terminate()